# Extra steps for displaying pdf through swf on doc88
**First thing first**, most of the underlying works were done by Winsen Jiansbomber (http://blog.csdn.net/WinsenJiansbomber/article/details/47065697?locationNum=1&fps=1). I just transformed part of his work into python. Honestly, my initial intention is to download one pdf from doc88 (which is clearly wrong). It turns out there are something interesting to learn.

According to Jimbo's research, the mechanism turns out very simple. Each page in a pdf was splitted into two parts: a ph file and a pk file. The ph file is shared by all pages within the pdf (probably not shared between different pdfs), and the pk file is page specific. The two files are labelled as .ebt file and are zlib compressed. Therefore, to get a usable swf from the two ebt files, steps to take are 1. uncompressing both ebt; 2. concatenating ph and pk, as shown by the code below: 

In [ ]:
import zlib
import struct

inphpath = 'ph.ebt'
inpkpath = 'pk.ebt'
outpath = 'out.swf'

phbytearray = open(inphpath, 'rb').read()
# ph has to be explicitely declared as mutable bytearray because it will be modified later for file length below.
ph = bytearray(zlib.decompress(phbytearray[40:]))
# This might not be necessary
ph[4:8] = struct.pack('<I',len(ph))

pkbytearray = open(inpkpath, 'rb').read()
# ph has to be explicitely declared as mutable bytearray because it will be modified later for file length below.
pk = bytearray(zlib.decompress(pkbytearray[32:]))

out = ph + pk + bytearray([64]) + bytearray([0]) + bytearray([0]) + bytearray([0])
out[4:8] = struct.pack('<I', len(out))

open(outpath, 'wb').write(out)


### 1 thing learned and 3 questions
As shown by [swf file format specs](http://wwwimages.adobe.com/content/dam/Adobe/en/devnet/swf/pdf/swf-file-format-spec.pdf), swf header has 6 fields. Just to mention 2 here:  
1. The first three bytes are swf signature. While the second and third are always WS, the first byte can indicates compression status (F: uncompressed; C: zlib compressed; Z: LZMA compressed). The signature bytes can be repurposed. For example, the ph file uses the signature bytes as "YBD".
2. The fifth to the eighth bytes are for file length. It should be uncompressed or decompressed SWF file size.

Questions (to be learned):
1. Since file length field is going to be reset in the end, setting the file length field for ph seems to be unnecessary.
2. What's the signature for zlib compressed file? How did Jimbo originally figure out ph and pk are zlib compressed? How to figure out that you will need to bypass 40 bytes for ph file and 32 bytes for pk file? There seems to be a rule because if it is not bypassed correctly (e.g. 41 or 39), decompress will fail.
3. What is the 4 bytes ending for? How do you know you will need that?